In [853]:
import pandas as pd
import altair as alt
alt.renderers.set_embed_options(theme='dark')


RendererRegistry.enable('default')

In [854]:
def claim(claim_hash, cols_claim):
    try:
        df_claim = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"],
        )
    except:
        return pd.DataFrame(columns = cols_claim[claim_hash])
    if(len(df_claim.columns)==0):
        return pd.DataFrame(columns = cols_claim[claim_hash])
    return df_claim

In [855]:
p2_users = '07f87a6f-5bc6-49f3-a1bb-8e7d545ba95a'
p2_hourly = 'de5232e1-5d57-4232-900d-1d09d2c9438e'

In [856]:
cols_claim = {
    p2_users: ["DEPOSIT_AMOUNT","HR","NET_DEPOSITED_AMOUNT","SENDER","WITHDRAWN_AMOUNT","WITHDRAWN_AMOUNT_PHASE2"],
    p2_hourly: ['HR', 'WITH_AMOUNT', 'WITH_TX', 'WITH_USERS']
}

In [857]:
user_stats = '499224b4-30a6-43d7-80b9-3a019cbb1d3d'
user_stats_df = claim(user_stats, cols_claim)

In [858]:
user_stats_df.columns = [c.lower() for c in user_stats_df.columns]
user_stats_df

,deposit_amount,deposit_txs,sender,withdrawn_amount,withdraw_txs
0,200.000,2,terra1u793r2hfd684ae3ymyrts3jgtfl5ur40j9f07t,0.0,0
1,150.000,1,terra15sxyjyzyz3k2gsyh8skp86lvtl7xj8fys5hrth,0.0,0
2,225.000,1,terra1n8rrqld0tgguqle30n2cg2xfxguuevhkf8kkth,0.0,0
3,1800.000,1,terra1t3gtrly507dw8fsxzqzyjs2zefm07a7ugkw0ka,0.0,0
4,96.896,1,terra1xhdpqx8laux3lgfm4amfqdtgyevuaxerezknye,0.0,0
...,...,...,...,...,...
5468,1000.000,1,terra17npp9fq9uzanv05rytgj86muaet59rpth55aln,0.0,0
5469,2500.000,1,terra10lwmun98vpv9fuup2jnjvwp70a9778ra8tqjpm,0.0,0
5470,100.000,1,terra1upp95s4gh0l7cpm7e5esw4kdpwwm68c8xnvtwh,0.0,0
5471,50.000,1,terra1f78fdgp380zq0n4fajv3vla0azu79z09kv7gza,0.0,0


In [859]:
def get_url(url):
    return pd.read_csv(url, index_col=0)

In [860]:
with_phase1 = get_url('https://raw.githubusercontent.com/IncioMan/prism_forge/master/data/with_phase1.csv')
with_phase1

,sender,deposit,total_deposit,withdrawable_amount,tokens_to_claim,can_claim
0,terra1u793r2hfd684ae3ymyrts3jgtfl5ur40j9f07t,200.000,2.505094e+07,183.333333,558.861168,False
1,terra15sxyjyzyz3k2gsyh8skp86lvtl7xj8fys5hrth,150.000,2.505094e+07,137.499999,419.145876,False
2,terra1n8rrqld0tgguqle30n2cg2xfxguuevhkf8kkth,225.000,2.505094e+07,206.249999,628.718814,False
3,terra1t3gtrly507dw8fsxzqzyjs2zefm07a7ugkw0ka,1800.000,2.505094e+07,1649.999999,5029.750514,False
4,terra1xhdpqx8laux3lgfm4amfqdtgyevuaxerezknye,96.896,2.505094e+07,88.821333,270.757058,False
...,...,...,...,...,...,...
5468,terra17npp9fq9uzanv05rytgj86muaet59rpth55aln,1000.000,2.503676e+07,875.000000,2795.888723,False
5469,terra10lwmun98vpv9fuup2jnjvwp70a9778ra8tqjpm,2000.000,2.503676e+07,0.000000,5591.777446,False
5470,terra1upp95s4gh0l7cpm7e5esw4kdpwwm68c8xnvtwh,100.000,2.503676e+07,87.500000,279.588872,False
5471,terra1f78fdgp380zq0n4fajv3vla0azu79z09kv7gza,50.000,2.503676e+07,43.750000,139.794436,False


In [861]:
p2_users_df = claim(p2_users, cols_claim)
p2_hourly_df = claim(p2_hourly, cols_claim)

In [862]:
p2_hourly_df

,HR,WITH_AMOUNT,WITH_TX,WITH_USERS
0,02/05 09:00,13504.749994,14,14
1,02/05 07:00,57928.803221,35,35
2,02/05 10:00,1200.000000,2,2
3,02/05 08:00,121191.269562,19,19


In [863]:
p2_users_df.columns = [c.lower() for c in p2_users_df.columns]
p2_users_df

,deposit_amount,hr,net_deposited_amount,sender,withdrawn_amount,withdrawn_amount_phase2
0,1170.163571,2022-02-05 08:00:00.000,400.163571,terra1pk5n5fg23wegksfh6r3kme42xk22dj5hq4h58h,770.0,383.490088
1,10000.000000,None,10000.000000,terra1xz24858m7ke9j7gnrkk0lat8utr3vjexckz0zg,0.0,NaN
2,500.000000,None,500.000000,terra1udmszv3fnqn87vv0t5f2z35enlxgw5yrsu8dxx,0.0,NaN
3,1244.630383,None,1244.630383,terra157tn3kzuemmkn8jpqkav7gh2ulmrllcg70y8hc,0.0,NaN
4,1.489442,None,1.489442,terra18xct67xk5jazp85r42s67vvlkfynpad4zj667l,0.0,NaN
...,...,...,...,...,...,...
5468,20000.000000,None,20000.000000,terra1futa5pnnwtxgk9n87e35s634plhaqe8rf8w67v,0.0,NaN
5469,10000.000000,None,10000.000000,terra1p4jpt6mcnvqptln5w22xlqwm6udsxg4eap3rkl,0.0,NaN
5470,75.000000,None,75.000000,terra19h89w0va3j00r5z2jnl80yy86apt3w0w3kkc0k,0.0,NaN
5471,50.000000,None,50.000000,terra1w3vgtrazlfsaevnwh0rxnagctas8qu4fgmw0qn,0.0,NaN


In [864]:
users_with = p2_users_df.merge(with_phase1,on='sender')[['sender','net_deposited_amount','deposit','withdrawable_amount']]
users_with=users_with.rename(columns={'net_deposited_amount':'deposited_p1'})
users_with['WITHDRAWN_AMOUNT_PHASE2'] = users_with.deposited_p1-users_with.deposit
users_with['has_withdrawn_p2'] = (users_with['WITHDRAWN_AMOUNT_PHASE2'])>1
users_with

,sender,deposited_p1,deposit,withdrawable_amount,WITHDRAWN_AMOUNT_PHASE2,has_withdrawn_p2
0,terra1pk5n5fg23wegksfh6r3kme42xk22dj5hq4h58h,400.163571,16.673483,0.000000,383.490088,True
1,terra1xz24858m7ke9j7gnrkk0lat8utr3vjexckz0zg,10000.000000,10000.000000,8750.000000,0.000000,False
2,terra1udmszv3fnqn87vv0t5f2z35enlxgw5yrsu8dxx,500.000000,500.000000,437.500000,0.000000,False
3,terra157tn3kzuemmkn8jpqkav7gh2ulmrllcg70y8hc,1244.630383,1244.630383,1089.051585,0.000000,False
4,terra18xct67xk5jazp85r42s67vvlkfynpad4zj667l,1.489442,1.489442,1.303261,0.000000,False
...,...,...,...,...,...,...
5468,terra1futa5pnnwtxgk9n87e35s634plhaqe8rf8w67v,20000.000000,20000.000000,17500.000000,0.000000,False
5469,terra1p4jpt6mcnvqptln5w22xlqwm6udsxg4eap3rkl,10000.000000,10000.000000,8750.000000,0.000000,False
5470,terra19h89w0va3j00r5z2jnl80yy86apt3w0w3kkc0k,75.000000,75.000000,65.625000,0.000000,False
5471,terra1w3vgtrazlfsaevnwh0rxnagctas8qu4fgmw0qn,50.000000,50.000000,43.750000,0.000000,False


In [865]:
users_with[users_with.has_withdrawn_p2]

,sender,deposited_p1,deposit,withdrawable_amount,WITHDRAWN_AMOUNT_PHASE2,has_withdrawn_p2
0,terra1pk5n5fg23wegksfh6r3kme42xk22dj5hq4h58h,400.163571,16.673483,0.0,383.490088,True
164,terra125908lwaf3tpyzqecwv6asvcjtfaj0kg86hsa3,2546.000000,0.000000,0.0,2546.000000,True
165,terra1j7vz2vje28m4wdqxxprvpg5nfnf5hzn2e0stdh,1200.000000,650.000000,0.0,550.000000,True
166,terra1h29qhduklvk00g0rsalcu9jk82cyyrsgydfmk5,1000.000000,0.000000,0.0,1000.000000,True
167,terra14cvtp373x6auhdzu77wczdd2y2l9wm9uwzf6zt,300.000000,0.000000,0.0,300.000000,True
...,...,...,...,...,...,...
4111,terra1005r2u35mrw9zqvn7j5d3x04gq6cap232kjwuw,1000.000000,83.333334,0.0,916.666666,True
4112,terra1lph05z2vr88n030pwnfmwacnl2ct5nzyhc9eyf,250.000000,180.000000,0.0,70.000000,True
4113,terra1xwk5vzknyula2shhd6y9ltyl07e0pch3jtesgm,150.000000,100.000000,0.0,50.000000,True
4114,terra1xpdlpqvswk2xs88e02kl8ahnxe73hhcmvmyr53,190.000000,7.916667,0.0,182.083333,True


In [866]:
cols_dict = {
    "DEPOSIT_AMOUNT":"Amount deposited",
    "DEPOSIT_TXS":"N° of deposit txs",
    "DEP_USERS":"N° of depositing users",
    "HR":"Time",
    "NET_AMOUNT":"Net amount deposited",
    "TOT_TXS":"N° of txs",
    "TOT_USERS":"N° of users",
    "WITH_AMOUNT":"Withdrawn amount",
    "WITH_TXS":"N° of withdraw txs",
    "WITH_USERS":"N° of withdrawing users",
    'AMOUNT':'Amount deposited (UST)',
    'AVG_BALANCE_USD':'Average balance in the last 30 days (UST)',
    'N_TXS':'N° of txs',
    'SENDER':'User address',
    'N_USERS':'N° of users',
    'BUCKET':'Deposit range',
    'bucket_name':'Deposit bucket',
    'cumsum_ust':'Cumulative UST amount',
    'MIN_DATE':'Date wallet creation',
    'ADDRESS_COUNT':'Number of wallets',
    'PERC_WITHDRAWN':'% withdrawn of the deposited amount',
    'net_ust': 'Amount UST',
    'perc_withdrawn_cat':'Percentage withdrawn UST bucket',
    'DEP_CAT':'Deposited UST range'
}

In [867]:
#Total withdrawn phase 2 UST
users_with['WITHDRAWN_AMOUNT_PHASE2'] = users_with['WITHDRAWN_AMOUNT_PHASE2'].apply(lambda x: x if x >0 else 0)
tot_with_ust = users_with['WITHDRAWN_AMOUNT_PHASE2'].sum()
tot_with_ust

194678.82277700002

In [868]:
#Total net dep phase 1 UST
tot_ust_p1 = users_with['deposited_p1'].sum()
tot_ust_p1

25242163.128493

In [869]:
#Total left UST
tot_net_ust = users_with.deposit.sum()
tot_net_ust

25047484.305716

In [870]:
#% withdrawn
perc_with_p2 = tot_with_ust/tot_ust_p1 * 100
perc_with_p2

0.7712446107966449

In [871]:
#current price
curr_price = tot_net_ust/70000000
curr_price

0.3578212043673714

In [872]:
#% users with
p_users_with_p2 = (users_with.has_withdrawn_p2.sum()/len(p2_users_df))*100
p_users_with_p2

1.3155490590169925

In [873]:
p2_users_df.head()

,deposit_amount,hr,net_deposited_amount,sender,withdrawn_amount,withdrawn_amount_phase2
0,1170.163571,2022-02-05 08:00:00.000,400.163571,terra1pk5n5fg23wegksfh6r3kme42xk22dj5hq4h58h,770.0,383.490088
1,10000.000000,None,10000.000000,terra1xz24858m7ke9j7gnrkk0lat8utr3vjexckz0zg,0.0,NaN
2,500.000000,None,500.000000,terra1udmszv3fnqn87vv0t5f2z35enlxgw5yrsu8dxx,0.0,NaN
3,1244.630383,None,1244.630383,terra157tn3kzuemmkn8jpqkav7gh2ulmrllcg70y8hc,0.0,NaN
4,1.489442,None,1.489442,terra18xct67xk5jazp85r42s67vvlkfynpad4zj667l,0.0,NaN


In [874]:
df = pd.DataFrame([[tot_ust_p1-tot_with_ust,tot_with_ust],['Still deposited','Withdrawn']]).T
df.columns = ['UST','Type']
df

,UST,Type
0,2.50475e+07,Still deposited
1,194679,Withdrawn


In [875]:
alt.Chart(df).mark_arc().encode(
    theta=alt.Theta(field="UST", type="quantitative"),
    color=alt.Color(field="Type", type="nominal",
            scale=alt.Scale(scheme='pastel1'),
            legend=alt.Legend(
            orient='none',
            padding=10,
            legendY=-10,
            direction='vertical'))
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [876]:
#UST left to be withdrawn
left_to_with = users_with.withdrawable_amount.sum()
left_to_with

21857074.21968

In [877]:
tot_net_ust - left_to_with

3190410.0860360004

In [881]:
tot_net_ust

25047484.305716

In [882]:
left_to_with

21857074.21968

In [884]:
(25036561 - 20754619)/70000000

0.0611706

In [878]:
#Floor price
floor_price = (tot_net_ust - left_to_with)/70000000
floor_price

0.04557728694337143

In [833]:
p2_hourly_df['HR'] = '2022/' + p2_hourly_df['HR'] + ':00'
p2_hourly_df.head()

,HR,WITH_AMOUNT,WITH_TX,WITH_USERS
0,2022/02/05 08:00:00,119998.948418,15,15
1,2022/02/05 07:00:00,57928.803221,35,35


In [834]:
alt.Chart(p2_hourly_df.rename(columns=cols_dict)).mark_bar().encode(
    x=cols_dict['HR']+':T',
    y=cols_dict['WITH_AMOUNT']+":Q",
    tooltip=[cols_dict['HR']+':T',cols_dict['WITH_AMOUNT']+":Q"]
).configure_mark(
    color='#fab0ba'
).properties(width=700).configure_axisX(
    labelAngle=30
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [835]:
alt.Chart(p2_hourly_df.rename(columns=cols_dict)).mark_bar().encode(
    x=cols_dict['HR']+':T',
    y=cols_dict['WITH_USERS']+":Q",
    tooltip=[cols_dict['HR']+':T',cols_dict['WITH_USERS']+":Q"]
).configure_mark(
    color='#fab0ba'
).properties(width=700).configure_axisX(
    labelAngle=30
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [836]:
users_with.WITHDRAWN_AMOUNT_PHASE2 = users_with.WITHDRAWN_AMOUNT_PHASE2.apply(lambda x: x if x >=0 else 0)

In [837]:
df = (round(with_users_df[with_users_df.has_withdrawn_p2]['WITHDRAWN_AMOUNT_PHASE2'],3)/with_users_df['deposited_p1']*10)
df[df<1]

1031    0.740741
dtype: float64

In [838]:
with_users_df = users_with[users_with.has_withdrawn_p2&users_with['deposited_p1']>0]
with_users_df['perc_withdrawn'] = round(with_users_df[with_users_df.has_withdrawn_p2]['WITHDRAWN_AMOUNT_PHASE2'],3)/with_users_df['deposited_p1']*10
with_users_df['perc_withdrawn_cat'] = (with_users_df['perc_withdrawn']).apply(lambda x: int(x) if int(x) < 10 else 9)
df2 = with_users_df.groupby('perc_withdrawn_cat').sender.count()
df2

<ipython-input-838-e0c469897b63>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_users_df['perc_withdrawn'] = round(with_users_df[with_users_df.has_withdrawn_p2]['WITHDRAWN_AMOUNT_PHASE2'],3)/with_users_df['deposited_p1']*10
<ipython-input-838-e0c469897b63>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_users_df['perc_withdrawn_cat'] = (with_users_df['perc_withdrawn']).apply(lambda x: int(x) if int(x) < 10 else 9)


perc_withdrawn_cat
0     1
1     1
2     1
3     3
4     3
5    10
6     7
7     4
8     6
9    41
Name: sender, dtype: int64

In [839]:
perc_cat = list(range(0,10))
cat = pd.DataFrame([0]*10,perc_cat)
df3 = cat.join(df2,how='outer')
df3

,0,sender
0,0,1
1,0,1
2,0,1
3,0,3
4,0,3
5,0,10
6,0,7
7,0,4
8,0,6
9,0,41


In [840]:
perc_cat = list(range(0,10))
cat = pd.DataFrame([0]*10,perc_cat)
df3 = cat.join(df2,how='outer')
df3.index =  ['0%-10%','10%-20%','20%-30%','30%-40%','40%-50%','50%-60%','60%-70%','70%-80%','80%-90%','90%-100%']
df3 = df3.sender.fillna(0).reset_index()
with_perc_buckets=df3.rename(columns={'index':'PERC_WITHDRAWN','sender':'TOT_USERS'})

In [841]:
with_perc_buckets

,PERC_WITHDRAWN,TOT_USERS
0,0%-10%,1
1,10%-20%,1
2,20%-30%,1
3,30%-40%,3
4,40%-50%,3
5,50%-60%,10
6,60%-70%,7
7,70%-80%,4
8,80%-90%,6
9,90%-100%,41


In [842]:
alt.Chart(with_perc_buckets.sort_values(by='PERC_WITHDRAWN').rename(columns=cols_dict)).mark_bar().encode(
    y=alt.X(cols_dict['PERC_WITHDRAWN']+":N", sort=alt.EncodingSortField(order='ascending')),
    x=cols_dict['TOT_USERS']+":Q",
    tooltip=[cols_dict['PERC_WITHDRAWN']+":N",cols_dict['TOT_USERS']+":Q"]
).configure_mark(
    color='#F1705F'
).properties(height=300).configure_axisX(
    labelAngle=0
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [843]:
p2_hourly_df = p2_hourly_df.sort_values(by='HR')
p2_hourly_df['cumsum_with'] = p2_hourly_df.sort_values(by='HR').WITH_AMOUNT.cumsum()
p2_hourly_df['net_ust'] = tot_ust_p1 - p2_hourly_df['cumsum_with']
p2_hourly_df.head()

,HR,WITH_AMOUNT,WITH_TX,WITH_USERS,cumsum_with,net_ust
1,2022/02/05 07:00:00,57928.803221,35,35,57928.803221,2.151747e+07
0,2022/02/05 08:00:00,119998.948418,15,15,177927.751639,2.139747e+07


In [844]:
alt.Chart(p2_hourly_df.rename(columns=cols_dict)).mark_line(point=True).encode(
    x=alt.X(cols_dict['HR']+':T', sort=alt.EncodingSortField(order='ascending')),
    y=cols_dict['net_ust']+":Q",
    tooltip=[alt.Tooltip(cols_dict['HR']+':T', format='%Y-%m-%d %H:%M'), alt.Tooltip(cols_dict['net_ust']+":Q")]
).configure_mark(
    color='#F1705F'
).properties(width=700).configure_axisX(
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [845]:
with_users_df[with_users_df['perc_withdrawn_cat']>8]

,sender,deposited_p1,deposit,withdrawable_amount,WITHDRAWN_AMOUNT_PHASE2,has_withdrawn_p2,perc_withdrawn,perc_withdrawn_cat
101,terra1h5uvce0xfseaf7x4cahyfdfqa3z7d8fqemljjw,44.161194,1.840050,0.0,42.321144,True,9.583301,9
192,terra16affkjl4tlszpv5xj369qnc96r8utrshw7qnrg,10.000000,0.833334,0.0,9.166666,True,9.167000,9
233,terra1ytk0hphc86fl256g5l5s9ea4cqspdgskctdtgh,200.000000,0.000000,0.0,200.000000,True,10.000000,9
305,terra130ef5vgxn77t0yzv2hmg9400qfjlwkklmuvhkh,600.000000,0.000000,0.0,600.000000,True,10.000000,9
330,terra1kzl6qd4hj30w9arxh3nc6zht259jwm4y0ayy26,1200.000000,100.000001,0.0,1099.999999,True,9.166667,9
371,terra12nskn56mcenhe6lvuuzxzhdjstk8yy6pa3spu4,250.000000,0.000000,0.0,250.000000,True,10.000000,9
475,terra1uscncgg7kcea2f07l2r2l8g75kda2hcpvz9d3v,500.000000,0.000000,0.0,500.000000,True,10.000000,9
587,terra1p9p2rnfhrt45gtfmrxj3t06kr5zdzak26na334,496.000000,0.000000,0.0,496.000000,True,10.000000,9
1004,terra12j3wtau0fu88fvnarjwc9kvgel36cu66ku4vgz,1000.000000,83.333334,0.0,916.666666,True,9.166670,9
1100,terra1jk4r2xxj6myl9vmsu9wvl498m9hf3mw9t9770c,1257.737000,0.000000,0.0,1257.737000,True,10.000000,9


In [846]:
with_users_df['perc_withdrawn_cat'].apply(lambda x: int(int(x*10/5)+2))

101     20
192     20
194     10
233     20
305     20
        ..
4919    18
5037     6
5100    20
5208    16
5230    16
Name: perc_withdrawn_cat, Length: 77, dtype: int64

In [847]:
with_users_df['DEP_CAT'] = (with_users_df['deposited_p1']/20000).apply(int)
with_users_df['perc_withdrawn_cat_old'] = with_users_df['perc_withdrawn_cat']
with_users_df['perc_withdrawn_cat'] = with_users_df['perc_withdrawn_cat'].apply(lambda x: int(int(x*10/5))+2)
df = with_users_df.groupby(['DEP_CAT','perc_withdrawn_cat']).sender.count()
df = df.reset_index()
df

<ipython-input-847-3e5616d1f5a4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_users_df['DEP_CAT'] = (with_users_df['deposited_p1']/20000).apply(int)
<ipython-input-847-3e5616d1f5a4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_users_df['perc_withdrawn_cat_old'] = with_users_df['perc_withdrawn_cat']
<ipython-input-847-3e5616d1f5a4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

,DEP_CAT,perc_withdrawn_cat,sender
0,0,4,1
1,0,6,1
2,0,8,2
3,0,10,3
4,0,12,6
5,0,14,6
6,0,16,3
7,0,18,6
8,0,20,39
9,1,12,3


In [848]:
alt.Chart(df).mark_rect().encode(
    x=alt.X('perc_withdrawn_cat:O', sort=alt.EncodingSortField(order='ascending')),
    y=alt.Y('DEP_CAT:O', sort=alt.EncodingSortField(order='descending')),
    color='sender:Q'
)

alt.Chart(...)

In [849]:
df

,DEP_CAT,perc_withdrawn_cat,sender
0,0,4,1
1,0,6,1
2,0,8,2
3,0,10,3
4,0,12,6
5,0,14,6
6,0,16,3
7,0,18,6
8,0,20,39
9,1,12,3


In [799]:
import random
df1 = pd.DataFrame([list(range(1,21)),
                    ["0%-5%",	"5%-10%",	"10%-15%",	"15%-20%",	"20%-25%",	"25%-30%",	"30%-35%",	
"35%-40%",	"40%-45%",	"45%-50%",	"50%-55%",	"55%-60%",	"60%-65%",	
"65%-70%",	"70%-75%",	"75%-80%",	"80%-85%",	"85%-90%",	"90%-95%",	"95%-100%"]]).T
df2 = pd.DataFrame([list(range(0,29)),
                    ["0-20k", "20-40k", "40-60k", "60-80k", "80-100k", "100-120k", "120-140k", "140-160k", 
                     "160-180k", "180-200k","200-220k", "220-240k", "240-260k", "260-280k", "280-300k", "300-320k",
                     "320-340k",
                     "340-360k", 
                     "360-380k", "380-400k", "400-420k", "420-440k", "440-460k", "460-480k", "480-500k", "500-520k","520-540k",
                     "540-560k","560-580k"]]).T
df2['fake'] = 0
df1['fake'] = 0
heatmap_val = df2.merge(df1,on='fake').drop('fake',1)
heatmap_val.columns = ['DEP_CAT','DEP_CAT_label','perc_withdrawn_cat','perc_withdrawn_cat_label']
heatmap_val

,DEP_CAT,DEP_CAT_label,perc_withdrawn_cat,perc_withdrawn_cat_label
0,0,0-20k,1,0%-5%
1,0,0-20k,2,5%-10%
2,0,0-20k,3,10%-15%
3,0,0-20k,4,15%-20%
4,0,0-20k,5,20%-25%
...,...,...,...,...
575,28,560-580k,16,75%-80%
576,28,560-580k,17,80%-85%
577,28,560-580k,18,85%-90%
578,28,560-580k,19,90%-95%


In [850]:
df_x = heatmap_val.merge(df, on=['perc_withdrawn_cat', 'DEP_CAT'], how='left').fillna(0)
heatmap_data_df = df_x.sort_values(by=['perc_withdrawn_cat', 'DEP_CAT'], ascending=[True,False])
heatmap_data_df[heatmap_data_df.DEP_CAT==1]

,DEP_CAT,DEP_CAT_label,perc_withdrawn_cat,perc_withdrawn_cat_label,sender
20,1,20-40k,1,0%-5%,0.0
21,1,20-40k,2,5%-10%,0.0
22,1,20-40k,3,10%-15%,0.0
23,1,20-40k,4,15%-20%,0.0
24,1,20-40k,5,20%-25%,0.0
25,1,20-40k,6,25%-30%,0.0
26,1,20-40k,7,30%-35%,0.0
27,1,20-40k,8,35%-40%,0.0
28,1,20-40k,9,40%-45%,0.0
29,1,20-40k,10,45%-50%,0.0


In [851]:
heatmap_data_df[heatmap_data_df.sender>0]

,DEP_CAT,DEP_CAT_label,perc_withdrawn_cat,perc_withdrawn_cat_label,sender
41,2,40-60k,2,5%-10%,1.0
3,0,0-20k,4,15%-20%,1.0
5,0,0-20k,6,25%-30%,1.0
47,2,40-60k,8,35%-40%,1.0
7,0,0-20k,8,35%-40%,2.0
9,0,0-20k,10,45%-50%,3.0
71,3,60-80k,12,55%-60%,1.0
31,1,20-40k,12,55%-60%,3.0
11,0,0-20k,12,55%-60%,6.0
73,3,60-80k,14,65%-70%,1.0


In [852]:
alt.Chart((heatmap_data_df.rename(columns=cols_dict))).mark_rect().encode(
    x=alt.X(cols_dict['perc_withdrawn_cat']+':O', sort=alt.EncodingSortField(order='ascending')),
    y=alt.Y(cols_dict['DEP_CAT']+':O', sort=alt.EncodingSortField(order='ascending')),
    color=alt.Color('sender'+':Q',
            scale=alt.Scale(scheme='redpurple')),
    tooltip=['DEP_CAT_label'+':N','perc_withdrawn_cat_label'+':N','sender'+':Q']
)

alt.Chart(...)